In [1]:
import random
import math
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import scipy
from scipy.optimize import minimize
from scipy.stats import gamma, norm
import seaborn as sns
import time

from functools import partial

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
N = int(1e5)  # MC iterations
def _r(x): return np.round(x, 2)
def _CI(mu, std, n=1, z=1): return mu + (z/np.sqrt(n)) * np.array([-std, std])

# Page Rank

In [3]:
P = np.array([[1, 0, 0, 0, 0, 0],
              [.5, 0, .5, 0, 0, 0],
              [0, 0, 0, 0, 1, 0],
              [0, 1/3, 1/3, 0, 1/3, 0],
              [0, 0, 0, 0, 0, 1],
              [0, 0, 0, 0, 1, 0]])

Look at $P^T$ because I want the **left** eigenvectors and eigenvalues of $P$ for the **stationary distribution**. (np.linalg.eig returns the right eigenvectors)

In [4]:
vals, vecs = np.linalg.eig(P.T)
vals

array([ 1., -1.,  1.,  0.,  0.,  0.])

Because we want to avoid spider traps, i.e. want to make the Markov Chain **irreducible** and **aperiodic**, we *jump* with probability $\alpha = 1/4$ uniformly on one of the nodes; thus, we'll have the matrix $U$ which is stochastic with 1/6 as all elements:
$$U = \frac{e e^T}{6}$$
with $e$ being the vector of ones.
$$P_{pagerank} = \alpha U + (1 - \alpha) P$$ 

In [5]:
alpha = 1/4
e = np.ones(len(P))
U = (1 / len(P)) * np.outer(e, e.T)

In [6]:
PR = alpha*U + (1-alpha)*P

In [7]:
vals, vecs = np.linalg.eig(PR.T)

Check importance of nodes:
- Assert first eigenvalue is one
- Grab the real part of first eigenvector (corresposnding to eigenvalue one) and normalize it.

In [8]:
assert(np.isclose(np.real(vals[0]), 1))
pi =  np.real(vecs[:,0] / np.sum(vecs[:,0]))
print('Stationary distribution:', pi)

Stationary distribution: [0.24479167 0.05208333 0.07161458 0.04166667 0.31324405 0.2765997 ]


Compute **return time** $\mu_i$. Once I know the stationary distribution, it's just:
$$\mu_i = \frac{1}{\pi_i}$$

Compute **hitting time** to $C^C = 3$. For all the states that are different than 3:
$$u(x) = 1 +  \sum_{y \in S} P(x, y) u(y)$$
$$u(3) = 0$$
When I use the second condition, I basically zero-out the transition probabilities to 3.


In [9]:
B = np.copy(PR)
B[:, 2] = 0
I = np.eye(len(B))
_r(np.linalg.solve(I - B, e)[2])

13.96